# Business Market Analysis
___

## Introduction / Business Problem

This analysis is meant to help a business venture choose between two locations. Specifically we will look at a Mexican retaraunt in Los Angeles. Finding a neighborhood that is not saturated with other Mexican restaunts would be ideal.

## Data

looking for market trends over multiple time series
* which neighborhoods have the fewest restauraunts
* number of current restaraunts and if the is an increse or decrease from previous periods
* an up and coming neghborhood would be fantastic
* Is the number of venues increasing or decreasing?
* Is the average rating for venues increasing or decreasing
* Can we pull census data to show population increase or decrease?
* Can we pull crime data in time series?
* Can we pull employment data time series?

In [ ]:
# Common imports I may need; will uncomment, add, etc as needed
import numpy as np
import pandas as pd
import folium
#from sklearn.cluster import KMeans
#import matplotlib.cm as cm
#import matplotlib.colors as colors
#import requests
#import urllib.request
#from urllib.request import urlopen
#import time
#from bs4 import BeautifulSoup

<div class="alert alert-block alert-danger">
<b>Delete this cell before publishing
</div>



### scrape for list of neighborhoods in LA


Population heatmaps would be a good source of data
* total population
* number of Mexican restaraunts

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_districts_and_neighborhoods_in_Los_Angeles" 
data = requests.get(url).text 
soup = BeautifulSoup(data,"html.parser")

In [ ]:
# This will need delimiters reset

# Create a list 
# Find the table
# Iterate
# Ignore "Not assigned"
# split, strip, clean, and label
# append list


table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [ ]:
# example if data shows up in an undesireable format
# This cell may not be necessary 
# make names easier to read
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

### find lat and long for each neighborhood


<div class="alert alert-block alert-info">
<b></b>This may be difficult if geocoder does not work
</div>

## define a function to locate nearby venues


In [ ]:
def getNearbyVenues(Neighborhood, Latitude, Longitude, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(Neighborhood, Latitude, Longitude):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(Neighborhood=df3['Neighborhood'],
                                   Latitude=df3['Latitude'],
                                   Longitude=df3['Longitude']
                                  )

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

In [ ]:
# how many nearby venues?
# May need to change limit
(toronto_venues.groupby(['Neighborhood']).size() 
   .sort_values(ascending=False) 
   .reset_index(name='count')) 

In [ ]:
# Classify venues by category
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
# Grouped by category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

# Choose category


Mexican Restaraunts

# list of venues in category


### Define a function to iterate through time series for each venue


In [ ]:
GET https://api.foursquare.com/v2/venues/timeseries


## Timeseries API Documentaion

### Authentication
User	required A valid OAuth access token in the query string of each request. Eg. &oauth_token=XXXX
___
### Parameters
venueId	AVNU234,AVNU567	A comma-separated list of venue ids to retrieve series data for. The current user must be the manager of all venues specified.

startAt	1284286794	required. The start of the time range to retrieve stats for (seconds since epoch).

endAt	1284286794	The end of the time range to retrieve stats for (seconds since epoch). If omitted, the current time is assumed.

fields	totalCheckins,newCheckins	Specifies which fields to return. May be one or more of totalCheckins, newCheckins, uniqueVisitors, sharing, genders, ages, hours, separated by commas.
___
### Response Fields
venueId	The venue id

date	An array of dates in the format yyyymmdd.

totalCheckins	An array of total check-in counts, one item for each day in the given range (inclusive).

newCheckins	An array of counts of new users visiting the venue, one item for each day in the given range (inclusive).

shareTwitter	An array of counts of check-ins shared to Twitter.

shareFacebook	An array of counts of check-ins shared to Facebook.

male	An array of counts of check-ins where the user has specified their gender as male.

female	An array of counts of check-ins where the user has specified their gender as female.

ageXX-XX	An array of counts of check-ins where the user has specified their age in the given range. The provided ranges are 13-17, 18-24, 25-34, 35-44, 45-54, and 55+.
___

## Is the number of venues increasing or decreasing?


## Is the average rating for venues increasing or decreasing

## Can we pull census data to show population increase or decrease?

## Can we pull crime data in time series?

## Can we pull employment data time series?